In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from dataPath import DATA_PATH
%matplotlib inline

import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,20)
matplotlib.rcParams["image.cmap"]="gray"

In [2]:
# OpenCV Utility Class for Mouse Handling
class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()
        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0])
        cv.imshow(self.windowname + ": mask", self.dests[1])

    # onMouse function for Mouse Handling
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None

        if self.prev_pt and flags & cv.EVENT_FLAG_LBUTTON:
            for dst, color in zip(self.dests, self.colors_func()):
                cv.line(dst, self.prev_pt, pt, color, 5)
            self.dirty = True
            self.prev_pt = pt
            self.show()

In [3]:
im = cv.imread(DATA_PATH+"images/Lincoln.jpg",cv.IMREAD_COLOR)
img_mask = im.copy()

inpaintMask = np.zeros(im.shape[:2],np.uint8)

sketch = Sketcher("image",[img_mask,inpaintMask],lambda :((255,255,255),255))
sketch.colors_func()

((255, 255, 255), 255)

In [ ]:
while True:
    ch = cv.waitKey()
    if ch == 27:
        break
    if ch == ord('t'):
        res = cv.inpaint(src=img_mask,inpaintMask=inpaintMask,inpaintRadius=3,flags=cv.INPAINT_TELEA)
        cv.imshow("Inpaint Output using FMM",res)
    if ch == ord('n'):
        res = cv.inpaint(src=img_mask,inpaintMask=inpaintMask,inpaintRadius=3,flags=cv.INPAINT_NS)
        cv.imshow("Inpaint Output using NS",res)

    if ch == ord('r'):
        img_mask[:]=im
        inpaintMask[:] = 0
        sketch.show()

cv.destroyAllWindows()